In [1]:
#Imports
import json
import numpy as np
import pandas as pd
import csv
import os

In [2]:
#encodes a dictionary into a jason
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def jasonGenerator(path,results,photo_name):
    """ Generates a .jason file in the path given folder from the a dictionary """
    with open(path + '/analysis_result_' + photo_name + '.json',"w") as miarch:
        miarch.write(json.dumps(results, cls=NumpyEncoder))

In [3]:
def jasonReader(path):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open(path,'r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [4]:
def slicesParser_max_peaksI(dictres):
#extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x) for x in range(0,len(allSlices[0]['max_peaks_intensities']))]    
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    for i in range(0,len(allSlices)):
        df_sum.loc[i] = allSlices[i]['max_peaks_intensities'][0:] 
    return df_sum

In [5]:
def slicesParser_halfpeakstime(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_half_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_half_peaks']
    return df_sum

In [6]:
def slicesParser_amplitudes(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['amplitudes']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['amplitudes']

    return df_sum

In [7]:
def slicesParser_min_peaksI(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['min_peaks_intensities']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['min_peaks_intensities']

    return df_sum

In [8]:
def slicesParser_peaktime(dictres):
    #extract all the slices data
    allSlices = dictres['slices']

    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_peaks']

    return df_sum

# Calcula el alternance ratio

AR_list = []
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
jsons_list = os.listdir(path)
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    slices = dic['slices']
    i = 0
    df = pd.DataFrame(columns=['transient1','transient2','transient_max'])
    print (file)
    for i in range(0,33):
        try:
            amps = slices[i]['amplitudes'][0:2]
            amps.append(max(amps))
            i += 1
            df.loc[len(df)] = amps
        except IndexError:
            pass
    df.to_csv(path + file + 'max_amp.csv', sep='\t')
print('Done')

# Guarda en tablas separadas los datos de cada json por parámetro

param_list = ['max_peaks_positions','min_peaks_positions','max_peaks_intensities','min_peaks_intensities','amplitudes','times_to_peaks','times_to_half_peaks']

for param in param_list:
    os.makedirs(path2 + param + '/')
    path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
    path2 = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/'
    jsons_list = os.listdir(path)
    for file in jsons_list:
        path_file = path + file
        dic = jasonReader(path_file)
        slices = dic['slices']
        i = 0
        df = pd.DataFrame(columns=['transient1','transient2'])
        print (file)
        for i in range(0,33):
            try:
                amps = slices[i][param][0:2]
                i += 1
                df.loc[len(df)] = amps
            except IndexError:
                pass
        df.to_csv(path2 + param + '/' + file + param + '.csv', sep='\t')
    print('Done')

# Guarda en una tabla los datos de la célula entera

path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
path2 = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/'
jsons_list = os.listdir(path)
os.makedirs(path2 + 'whole_cell/')
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['image']
    pairs = {k: dic[k] for k in list(dic)[1:]}
    keys = list(pairs.keys())
    df = pd.DataFrame()
    for label in range(0,len(pairs)):
        df[keys[label]] = pairs[keys[label]][0:2]
    df = df.T
    df.columns = ['Transient1', 'Transient2']
    df.to_csv(path2 + 'whole_cell/' + file + '.csv', sep='\t')
print('Done')

In [51]:
# Cálculo del TAU67%

TAU = pd.DataFrame()
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
path2 = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/'
jsons_list = os.listdir(path)
# os.makedirs(path2 + 'whole_cell/')
for file in jsons_list[0:1]:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['slices']
    for peak in range  (0, len(dic)):
        amp = ((np.array(dic[peak]['max_peaks_intensities'][0:2]) - np.array(dic[peak] ['min_peaks_intensities'][1:3]))/np.exp(1))+np.array(dic[peak] ['min_peaks_intensities'][1:3])
        for i in range(0,len(amp)):
            trans_list = dic[peak]['intensities'][dic[peak]['max_peaks_positions'][i]:dic[peak]['min_peaks_positions'][i+1]]
            print(dic[peak]['max_peaks_positions'][i],trans_list.index(int(amp[i])))
#             ttp50 = (np.abs(lista_up - amp50[i])).argmin()
#             lista_ttp50.append (ttp50)
#         tiempo_50pico ['Pico_' + str(peak)] = lista_ttp50

050320c8_analysis_result_8b000.json
61 63
295 51
60 64
296 58
61 63
295 56
61 64
295 56
60 62
295 56
62 61
296 55
61 62
294 57
62 65
296 55
61 66
295 56
62 65
296 58
62 64
296 53
62 65
296 55
63 60
297 55
65 59
298 55
66 55
299 53
63 54
297 56
62 54
296 52
62 55
297 51
61 60
296 55
62 60
300 52
62 60
298 54
62 59
297 56
61 61
297 59
63 61
298 58
62 64
296 57
62 65
296 55
62 69
296 59
62 66
296 52
62 67
296 55
61 61
296 57
61 61
296 57
60 63
296 59
61 64
295 59
60 68
296 55
61 62
295 57
61 64
295 58
61 69
297 58
61 73
297 60
62 66
297 58
62 65
296 53
61 59
296 49
61 54
296 50


In [ ]:
# Recalcula la amplitud normalizando por el baseline de 1Hz

